# I asked ChatGPT to "write a script to collect data from the USGS NWIS"

# The following cell contains a simple script to get a single variable from a single site

In [1]:
import requests
import pandas as pd

# set parameters
site_id = '01589300'  # replace with the USGS site ID you want to collect data for
start_date = '2020-01-01'
end_date = '2020-12-31'
parameter_code = '00060'  # streamflow in cubic feet per second

# make API request
url = f'https://waterservices.usgs.gov/nwis/dv/?format=json&sites={site_id}&startDT={start_date}&endDT={end_date}&parameterCd={parameter_code}'
response = requests.get(url)

# parse JSON response into a DataFrame
data = response.json()['value']['timeSeries'][0]['values'][0]['value']
df = pd.DataFrame(data)
df['dateTime'] = pd.to_datetime(df['dateTime'], format='%Y-%m-%d %H:%M:%S')
df.set_index('dateTime', inplace=True)
df['value'] = pd.to_numeric(df['value'])
df.rename(columns={'value': 'streamflow_cfs'}, inplace=True)

# print the first 5 rows of the DataFrame
print(df.head())

            streamflow_cfs qualifiers
dateTime                             
2020-01-01            31.8        [A]
2020-01-02            27.6        [A]
2020-01-03            36.4        [A]
2020-01-04            49.1        [A]
2020-01-05            34.8        [A]


# Search for parameter codes by keyword. You can also use: https://help.waterdata.usgs.gov/parameter_cd?group_cd=%

In [2]:
import requests
from bs4 import BeautifulSoup

# URL for the USGS parameter code webpage
url = "https://help.waterdata.usgs.gov/parameter_cd?group_cd=%"

# Get webpage content
response = requests.get(url)

# Parse webpage content with BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Create dictionary of parameter codes and names
parameter_dict = {}
for row in soup.find_all("tr")[1:]:
    cols = row.find_all("td")
    parameter_code = cols[0].text.strip()
    parameter_name = cols[2].text.strip()
    parameter_dict[parameter_code] = parameter_name

# Get user input
search_term = input("Enter a search term: ")

# Search for parameter codes that contain the search term
results = {}
for code, name in parameter_dict.items():
    if search_term.lower() in name.lower():
        results[code] = name

# Print results
if results:
    print(f"Results for '{search_term}':")
    for code, name in results.items():
        print(f"{code}: {name}")
else:
    print(f"No results found for '{search_term}'.")


Enter a search term: discharge
Results for 'discharge':
00060: Discharge, cubic feet per second
00061: Discharge, instantaneous, cubic feet per second
04122: Bedload sediment discharge, daily average, per unit width of river, composite samples, short tons per day per foot of width
30208: Discharge, cubic meters per second
30209: Discharge, instantaneous, cubic meters per second
50042: Discharge, gallons per minute
62856: Discharge, peak for storm event, cubic feet per second
72122: Discharge, median for storm event, cubic feet per second
72123: Discharge, mean for storm event, cubic feet per second
72137: Discharge, tidally filtered, cubic feet per second
72138: Discharge, tidally filtered, cubic meters per second
72139: Discharge, tidally filtered, gallons per minute
72177: Groundwater discharge, centimeters per day
72243: Discharge, cubic feet per day
72255: Mean water velocity for discharge computation, feet per second
72258: Coefficient used to adjust discharge, Slope-Q computation

In [3]:
search_term = input("Enter a search term: ")

# Search for parameter codes that contain the search term
results = {}
for code, name in parameter_dict.items():
    if search_term.lower() in name.lower():
        results[code] = name

# Print results
if results:
    print(f"Results for '{search_term}':")
    for code, name in results.items():
        print(f"{code}: {name}")
else:
    print(f"No results found for '{search_term}'.")

Enter a search term: ph
Results for 'ph':
00076: Turbidity, water, unfiltered, nephelometric turbidity units
00149: Alpha-emitting isotopes of radium, water, filtered, picocuries per liter
00294: Chemical oxygen demand, soluble, dry atmospheric deposition, milligrams per kilogram
00296: Chemical oxygen demand, insoluble, dry atmospheric deposition, milligrams per kilogram
00297: Chemical oxygen demand, total, dry atmospheric deposition, milligrams per kilogram
00400: pH, water, unfiltered, field, standard units
00403: pH, water, unfiltered, laboratory, standard units
00408: pH, water, filtered, laboratory, standard units
00410: Acid neutralizing capacity, water, unfiltered, fixed endpoint (pH 4.5) titration, field, milligrams per liter as calcium carbonate
00411: Acid neutralizing capacity, water, unfiltered, methyl orange endpoint (pH 3.1-4.4) titration, milligrams per liter as calcium carbonate
00415: Acid neutralizing capacity, water, unfiltered, phenolphthalein endpoint (pH 8.5-9.0

68464: Phosmet, core material, recoverable, dry weight, micrograms per kilogram
68499: 1-Naphthol, water, filtered, recoverable, nanograms per liter
68510: 3-Phenoxybenzyl alcohol, water, filtered, recoverable, nanograms per liter
68512: 4-Chloro-2-methylphenol, water, filtered, recoverable, nanograms per liter
68519: Acephate, water, filtered, recoverable, nanograms per liter
68521: 2-Chloro-N-(2-ethyl-6-methylphenyl)acetamide, water, filtered, recoverable, nanograms per liter
68531: alpha-Cypermethrin, water, filtered, recoverable, nanograms per liter
68532: alpha-Deltamethrin, water, filtered, recoverable, nanograms per liter
68546: 2-(4-tert-Butylphenoxy)-cyclohexanol, water, filtered, recoverable, nanograms per liter
68560: Dimethyl tetrachloroterephthalate mono-acid degradate (DCPA mono-acid), water, filtered, recoverable, nanograms per liter
68573: Dicrotophos, water, filtered, recoverable, nanograms per liter
68595: 2-[(2-Ethyl-6-methylphenyl)amino]-1-propanol, water, filtered,

# But we can do better than ChatGPT! Using some base elements from what ChatGPT gave us, let's build a function that allows us to get more data. With this, we can search all the sites for a particular state and look for as many variables as we want. We can also use multiprocessing to make the code run faster. It's still pretty slow though, so I'm sure there are more efficient ways to get data. But this should be fine for now...

In [4]:

#Define a function that we will call in our multiprocessing loop. This function should take one input that has a different value for each processor
def get_data(parameter_code):
    #We have to import and define stuff inside the function for multiprocessing.
    import requests
    import csv
    import pandas as pd
    import numpy as np
    #State code parameter
    state_code = 'NV'
    #start date parameter
    start_date = '1990-01-01'
    #end date parameter
    end_date = '2020-12-31'
# make API request for site data
    url = f'https://waterservices.usgs.gov/nwis/site/?format=rdb&stateCd={state_code}&parameterCd={parameter_code}'
    site_response = requests.get(url)
    all_data=[]
# Loop through the response we get from the URL, which contains all the sites in Nevada
    for i, row in enumerate(site_response.text.splitlines()):
        if row.split('\t')[0] == 'agency_cd':
            #Headers start where the first element of the row equals "agency_cd"
            headers = row.split('\t')
            #This means there is data here
        elif row.split('\t')[0] == 'USGS':
            site_info = row.split('\t')
            site_code = site_info[headers.index('site_no')]
            site_name = site_info[headers.index('station_nm')]
            latitude = site_info[headers.index('dec_lat_va')]
            longitude = site_info[headers.index('dec_long_va')]

            # make API request for variable data for each site
            try:
                dv_url = f'https://waterservices.usgs.gov/nwis/dv/?format=json&sites={site_code}&startDT={start_date}&endDT={end_date}&parameterCd={parameter_code}'
                dv_response = requests.get(dv_url)
                #Use the JSON format to parse the data (idk how this works)
                data = dv_response.json()['value']['timeSeries'][0]['values'][0]['value']
                df = pd.DataFrame(data)
                df['dateTime'] = pd.to_datetime(df['dateTime'], format='%Y-%m-%d %H:%M:%S')
                df.set_index('dateTime', inplace=True)
                df['value'] = pd.to_numeric(df['value'])
                df['latitude']=latitude
                df['longitude']=longitude
                df['site_no']=site_code
                df.drop(columns=['qualifiers'],inplace=True)
                #Append the site data to a list
                all_data.append(df)
                print ('Collecting data for ' + site_name + ' for parameter code '+parameter_code)
            except:
                site_data=0
    #time.sleep(10)
    try:
        #combine the data from each site
        concat_data=all_data[0]
        for i in range(1,len(all_data)):
            concat_data=pd.concat([concat_data,all_data[i]], axis=0)
        return concat_data
    except:
        print('error')
        return 'error'


#Import the parallelization library we will used
from ipyparallel import Client
#If you haven't installed ipyparallel, you will need to enter the following command in your terminal:
# pip install ipyparallel 
#We will also need to enter the following command into our terminal/anaconda prompt to start the ipyparallel cluster:
# ipcluster start -n 4
#Where 4 is the number of processors to use
rc = Client()
view = rc.load_balanced_view()
# Define which parameter codes you want to search for. Here we have discharge, temp, conductance and pH
parameter_codes = ['00060','00010', '00095']

async_results = []
#Loop through parameter codes
for p in parameter_codes:
    #apply your function to the parallel cluster. 1 process for each item you in the loop.
    async_result = view.apply_async(get_data, p)
    async_results.append(async_result)

rc.wait_interactive(async_results)

results = [ar.get() for ar in async_results]
print(results)

unknown:   0%|          | 0/3 [00:00<?, ?tasks/s]

[            value    latitude     longitude          site_no
dateTime                                                    
1992-10-01  59.00   36.728863  -114.2274714         09415190
1992-10-02  53.00   36.728863  -114.2274714         09415190
1992-10-03  53.00   36.728863  -114.2274714         09415190
1992-10-04  53.00   36.728863  -114.2274714         09415190
1992-10-05  62.00   36.728863  -114.2274714         09415190
...           ...         ...           ...              ...
2014-05-31   0.01  41.4268611  -119.1610556  412537119094001
2014-06-01   0.01  41.4268611  -119.1610556  412537119094001
2014-06-02   0.01  41.4268611  -119.1610556  412537119094001
2014-06-03   0.01  41.4268611  -119.1610556  412537119094001
2014-06-04   0.01  41.4268611  -119.1610556  412537119094001

[1527459 rows x 4 columns],             value     latitude     longitude          site_no
dateTime                                                     
2019-07-17   30.5   36.7955312   -114.069968         

In [5]:
import numpy as np
import pandas as pd
#Find the unique site numbers for each WQ variable (parameter code)
unique_sites0=np.unique(results[0].site_no)
unique_sites1=np.unique(results[1].site_no)
unique_sites2=np.unique(results[2].site_no)
#unique_sites3=np.unique(results[3].site_no)

#Concatenate all the uniqe sites
unique_sites=np.concatenate([unique_sites0,unique_sites1,unique_sites2])
unique, counts = np.unique(unique_sites, return_counts=True)
#Find sites where the unique count is 3 (contained in unique_site for each parameter, which means that site has all 3 parameters)
sites_with_all_data=unique[counts==3]
sites_with_all_data

array(['09415250', '09415850', '09416000', '09418700', '09419507',
       '094196783', '10301500', '10302002', '10302025', '10309000',
       '10311400', '10312000', '10312210', '1031221902', '10312275',
       '10333000', '10335000', '10336000', '10336035', '10336039',
       '10348200', '10348300', '10349980', '10351700'], dtype=object)

In [6]:
# If you have more than 4 parameters, just continue the pattern for the df, df_slice and df_concat definitions
# I haven't included pH here because there are only 2 sites with all 4 variables
df0=results[0]
df1=results[1]
df2=results[2]
#df3=results[3]


df0_slice=df0[df0.site_no == sites_with_all_data[0]]
df1_slice=df1[df1.site_no == sites_with_all_data[0]]
df2_slice=df2[df2.site_no == sites_with_all_data[0]]
#df3_slice=df3[df3.site_no == sites_with_all_data[0]]

df_concat=pd.concat([df1_slice, df0_slice.iloc[:,0:1],df2_slice.iloc[:,0:1]], axis=1).fillna(method='ffill').dropna()
df_concat.columns=(['temp', 'lat', 'long', 'site_no', 'discharge','conductivity'])
df_concat_final = df_concat
count=0
for i in sites_with_all_data:
    if count>0:
        df0_slice=df0[df0.site_no == i]
        df1_slice=df1[df1.site_no == i]
        df2_slice=df2[df2.site_no == i]
        
        df_concat=pd.concat([df1_slice, df0_slice.iloc[:,0:1],df2_slice.iloc[:,0:1]], axis=1).fillna(method='ffill').dropna()
        df_concat.columns=(['temp', 'lat', 'long', 'site_no', 'discharge','conductivity'])
        df_concat_final=pd.concat([df_concat_final,df_concat], axis=0)
    count=count+1


In [7]:
df_concat_final

,temp,lat,long,site_no,discharge,conductivity
dateTime,,,,,,
2008-02-16,14.5,36.51325,-114.3359806,09415250,219.0,3090.0
2008-02-17,15.5,36.51325,-114.3359806,09415250,207.0,2740.0
2008-02-18,14.5,36.51325,-114.3359806,09415250,212.0,2860.0
2008-02-19,14.5,36.51325,-114.3359806,09415250,193.0,2960.0
2008-02-20,15.0,36.51325,-114.3359806,09415250,190.0,2980.0
...,...,...,...,...,...,...
2020-12-27,2.8,39.7773722,-119.3375222,10351700,164.0,410.0
2020-12-28,2.6,39.7773722,-119.3375222,10351700,150.0,396.0
2020-12-29,3.4,39.7773722,-119.3375222,10351700,141.0,396.0


# Find all sites with more than 10,000 values for a particular variable. Currently Broken. Given the above code, this shouldn't be too tricky to fix though

In [8]:
import requests
import csv
from io import StringIO

# get user input for parameter code
state_code = 'NV'
parameter_code = '00060'  # discharge

# make API request for site data
url = f'https://nwis.waterdata.usgs.gov/nv/nwis/dv/?referred_module=sw&site_tp_cd=ST&site_tp_cd=ST-CA&site_tp_cd=ST-DCH&site_tp_cd=ST-TS&site_tp_cd=ST-SP&format=rdb&freq=D&stat_cds=00003,00010,00060,00065&startDT=1800-01-01&endDT=2023-03-06&parameterCd={parameter_code}&stateCd={state_code}&siteStatus=all'
site_response = requests.get(url)

# parse site data
sites = {}
for row in csv.reader(StringIO(site_response.text), delimiter='\t'):
    if len(row) == 1 and row[0].startswith('#'):
        continue
    elif row[0] == 'agency_cd':
        continue
    elif row[0] == 'site_no':
        site_code_index = row.index('site_no')
        latitude_index = row.index('dec_lat_va')
        longitude_index = row.index('dec_long_va')
        discharge_index = row.index('count_nu')
    else:
        site_code = row[site_code_index]
        latitude = row[latitude_index]
        longitude = row[longitude_index]
        discharge_count = int(row[discharge_index])
        if discharge_count > 10000:
            sites[site_code] = {'latitude': latitude, 'longitude': longitude, 'discharge_count': discharge_count}

# print data
for site_code, site_data in sites.items():
    print(site_code, site_data['latitude'], site_data['longitude'], site_data['discharge_count'])


NameError: name 'site_code_index' is not defined